# DB Tool Example

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import sqlite3
conn = sqlite3.connect("company.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

cur.executemany(
    "INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)",
    [
        ("Alice", "AI", 120000),
        ("Bob", "Data", 95000),
        ("Charlie", "Cloud", 110000),
        ("Diana", "AI", 130000),
        ("Ethan", "Data", 105000)
    ]
)
conn.commit()
conn.close()
print("Sample DB ready: company.db")


Sample DB ready: company.db


In [3]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, InfoSQLDatabaseTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents import AgentExecutor

# --- LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Database connection ---
db = SQLDatabase.from_uri("sqlite:///company.db")

# --- Tools ---
query_tool = QuerySQLDataBaseTool(db=db)
info_tool = InfoSQLDatabaseTool(db=db)
tools = [info_tool, query_tool]

# --- Prompt ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a data analyst assistant. "
     "Use InfoSQLDatabaseTool to inspect the schema when needed, "
     "and QuerySQLDataBaseTool to fetch data. "
     "Always explain your reasoning and show the final answer clearly."),
    MessagesPlaceholder("agent_scratchpad"),
    ("human", "{input}")
])

# --- Agent ---
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Example query ---
q = "Which department has the highest average salary, and what is that amount?"
result = executor.invoke({"input": q})
print("\n=== FINAL ANSWER ===\n", result["output"])


/var/folders/mb/cqx7lgmx7_9_rxg83s8jqy6h0000gn/T/ipykernel_84011/4289456855.py:15: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  query_tool = QuerySQLDataBaseTool(db=db)




> Entering new AgentExecutor chain...

Invoking: `sql_db_schema` with `{'table_names': 'employees'}`



CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	department TEXT, 
	salary INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	department	salary
1	Alice	AI	120000
2	Bob	Data	95000
3	Charlie	Cloud	110000
*/
Invoking: `sql_db_query` with `{'query': 'SELECT department, AVG(salary) AS average_salary FROM employees GROUP BY department ORDER BY average_salary DESC LIMIT 1;'}`


[('AI', 125000.0)]The department with the highest average salary is the **AI** department, with an average salary of **$125,000**.

> Finished chain.

=== FINAL ANSWER ===
 The department with the highest average salary is the **AI** department, with an average salary of **$125,000**.
